In [6]:
import findspark

In [7]:
findspark.init("/opt/manual/spark/")

In [8]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [9]:
spark = SparkSession.builder \
.appName("Execution Plan") \
.master("local[2]") \
.config("spark.driver.memory","1500m") \
.config("spark.executer.memory","2g") \
.getOrCreate()

In [11]:
df = spark.read \
.option("header", True) \
.option("inferSchema", True) \
.option("compression","gzip") \
.csv("file:///home/train/datasets/Hotel_Reviews.csv.gz")

In [12]:
df2 = df.withColumn("Tags", F.split(F.col("Tags"),",")
                   .cast(ArrayType(StringType()))) \
.withColumn("Review_Date", F.to_date(F.col("Review_Date"),"M/d/yyyy"))

In [13]:
df2.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683


In [15]:
df3 = df2.select("Review_Date", "Average_Score","Reviewer_Nationality") \
.withColumn("Year", F.year("Review_Date")) \
.groupBy("Year") \
.agg(F.sum("Average_Score").alias("Total_Average_Score"))

In [16]:
df3.explain(True)

== Parsed Logical Plan ==
'Aggregate ['Year], [unresolvedalias('Year, None), sum('Average_Score) AS Total_Average_Score#120]
+- Project [Review_Date#68, Average_Score#19, Reviewer_Nationality#21, year(Review_Date#68) AS Year#110]
   +- Project [Review_Date#68, Average_Score#19, Reviewer_Nationality#21]
      +- Project [Hotel_Address#16, Additional_Number_of_Scoring#17, to_date('Review_Date, Some(M/d/yyyy)) AS Review_Date#68, Average_Score#19, Hotel_Name#20, Reviewer_Nationality#21, Negative_Review#22, Review_Total_Negative_Word_Counts#23, Total_Number_of_Reviews#24, Positive_Review#25, Review_Total_Positive_Word_Counts#26, Total_Number_of_Reviews_Reviewer_Has_Given#27, Reviewer_Score#28, Tags#50, days_since_review#30, lat#31, lng#32]
         +- Project [Hotel_Address#16, Additional_Number_of_Scoring#17, Review_Date#18, Average_Score#19, Hotel_Name#20, Reviewer_Nationality#21, Negative_Review#22, Review_Total_Negative_Word_Counts#23, Total_Number_of_Reviews#24, Positive_Review#25, Rev

In [17]:
spark.stop()